In [2]:
import pandas as pd # Para el manejo de los datos
import re # Para expreciones regulares
import spacy # Para realiar el proceso de lematización

from unidecode import unidecode # Para eliminar acentos
from nltk.tokenize import word_tokenize # Para dividir el texto en tokens
from nltk.corpus import stopwords  # Para eliminar las palabras vacías
from nltk.stem import SnowballStemmer  # Para realizar el proceso de Stemming

## Se cargan los datos

In [3]:
df = pd.read_csv('Data/datos_clasificados_persona.csv', sep=';', encoding='utf-8')

## Se convierte el texto a minúsculas

In [4]:
df['minusculas'] = df['MENSAJES'].str.lower()

## Se eliminan los signos de puntuación y los acentos

In [5]:
reg_signos = re.compile('[^\w\s]')
df['puntuacion'] = df['minusculas'].replace(reg_signos, '')
df['acentos'] = df['puntuacion'].apply(unidecode)

## Se covierte el texto en tokens

In [6]:
df['tokens'] = df['acentos'].apply(word_tokenize)

In [7]:
df['tokens'][0]

['por',
 'favor',
 'pongamos',
 'en',
 'la',
 'portada',
 'algo',
 'que',
 'tenga',
 'relacion',
 'con',
 'la',
 'asignatura']

## Se eliminan las palabras vacias 

In [8]:
stop_words = set(stopwords.words('spanish'))
stop_words.add('tal')
df['stopwords'] = df['tokens'].apply(lambda x: [item for item in x if item not in stop_words])  

In [9]:
df['stopwords'][2]

['gusta', 'esperemos', 'haber', 'dicen', 'companeros', 'equipo']

## Se lematiza el texto

In [10]:
nlp = spacy.load('es_core_news_sm')

def lemmatizer(text):  
  doc = nlp(str(text))
  return ' '.join([word.lemma_ for word in doc])

df['lemas'] = df['stopwords'].apply(lambda x: lemmatizer(x))
df['lemas'] = df['lemas'].replace(reg_signos, '')
df['lemas'] = df['lemas'].apply(word_tokenize)

In [11]:
df['lemas'][2]

['gustar', 'esperemos', 'haber', 'decir', 'companero', 'equipo']

## Se realiza el proceso de Stemming

In [12]:
spanishstemmer = SnowballStemmer('spanish')
df['stemming'] = df['lemas'].apply(lambda tokens: [spanishstemmer.stem(token) for token in tokens])

## Se eliminan las filas que quedaron vacías

In [13]:
i = 0
for texto in df['stemming']:
    if len(texto) == 0:
       df = df.drop([i], axis=0)
    i += 1
df = df.reset_index(drop=True)

In [14]:
df['stemming'][1]

['buen', 'tard', 'imag', 'companer', 'algun', 'sugerent']

## Se guarda el texto proceso en un archivo csv

In [15]:
df_procesado = pd.DataFrame()
texto = [' '.join(texto) for texto in df['stemming']]
df_procesado['MENSAJES'] = texto
df_procesado['POLARIDAD'] = df['POLARIDAD']
df_procesado.to_csv('Data_procesada/datos_procesados.csv', sep=';', index=None)